In [16]:

import sys
from pathlib import Path
import preprocessing
from preprocessing import process_return_tables

import pandas as pd
from bs4 import BeautifulSoup
import re

from train import Trainer


from feature_engineering import FeatureCreator
from prediction import predict
from evaluation import Evaluator

DATA_DIR = Path("..","data")

COMMON_DATA_DIR = Path("..","..","common","data")
INPUT_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"


RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"

OUTPUT_DIR = Path("..","data","01_preprocessed" )
OUTPUT_DIR.mkdir(exist_ok = True,parents = True)


HTML_DIR = DATA_DIR/"html"
HTML_RACE_DIR = HTML_DIR / "race"
from feature_engineering import PredictionFeatureCreator
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# データ加工

# レース結果テーブルの加工

In [17]:
pd.read_csv(RAWDF_DIR /"results.csv",sep="\t")

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,owner_id,trainer_id
0,202206040804,1,4,7,ビターグラッセ,牝2,54.0,田辺裕信,2:02.6,NaN,7.1,3.0,502(+4),[東] 栗田徹,2020103228,1075,849030,1127
1,202206040804,2,6,12,ジュドー,牡2,55.0,菅原明良,2:02.7,1/2,4.5,2.0,474(-4),[東] 手塚貴久,2020103534,1179,506800,1038
2,202206040804,3,5,9,コスモバラタ,牡2,55.0,柴田大知,2:03.1,2.1/2,32.5,9.0,452(0),[東] 武市康男,2020105746,1009,716800,1089
3,202206040804,4,3,6,ブラックヴァール,牝2,54.0,丹内祐次,2:03.1,クビ,13.0,5.0,446(-6),[東] 菊川正達,2020100742,1091,553031,1052
4,202206040804,5,3,5,グランツグリーン,牡2,55.0,Ｍ．デム,2:03.1,アタマ,2.7,1.0,404(0),[東] 相沢郁,2020102191,5212,295007,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94887,202202010505,4,4,4,スノーパトロール,牝2,54.0,横山武史,1:11.0,1/2,7.3,3.0,480(0),[東] 上原博之,2020101817,1170,3060,423
94888,202202010505,5,8,8,ホーリーホック,牝2,54.0,丸山元気,1:11.7,4,25.6,5.0,466(0),[東] 大江原哲,2020106228,1127,351803,1012
94889,202202010505,6,3,3,タガノマカロン,牝2,54.0,丹内祐次,1:12.1,2.1/2,30.8,6.0,428(0),[西] 藤沢則雄,2020106036,1091,859002,1041
94890,202202010505,7,2,2,ビービータンバム,牡2,54.0,松田大作,1:12.6,3,85.9,8.0,436(0),[西] 千田輝彦,2020101124,1030,185800,1120


# 予測時の処理

In [1]:
# レース前日準備

In [18]:
import preprocessing

horse_results_preprocessed = preprocessing.process_horse_results(
    save_filename="horse_results_prediction.csv"
)

In [19]:
horse_results_preprocessed

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,win,rentai,show,place
0,0,2024-08-18,1.0,790.0,0.0,0.0,2,3000,0,1.0,12,195.8,1,1,1,7.0
1,1,2024-08-03,5.0,184.0,0.4,0.0,0,1800,0,4.0,15,112.3,0,0,0,4.0
2,2,2024-06-01,2.0,320.0,0.2,0.0,2,3000,2,1.0,10,205.4,0,1,1,5.0
3,3,2024-05-12,3.0,200.0,0.3,1.0,2,2890,0,1.0,11,189.5,0,0,1,4.0
4,4,2024-02-24,6.0,0.0,2.9,0.0,2,2860,1,1.0,10,196.4,0,0,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4082,12,2023-05-28,6.0,0.0,0.8,0.0,0,1600,0,2.0,16,97.5,0,0,0,5.0
4083,13,2023-02-19,10.0,0.0,2.2,1.0,0,1400,2,2.0,16,86.6,0,0,0,5.0
4084,14,2022-11-26,7.0,0.0,0.8,1.0,0,1600,2,5.0,10,97.4,0,0,0,5.0
4085,15,2022-11-05,1.0,520.0,0.0,1.0,0,1600,0,1.0,14,98.7,1,1,1,5.0


In [20]:
fc = FeatureCreator()
features = fc.create_features()

In [21]:
#重複チェック
features.duplicated(subset=["race_id","horse_id"]).sum()

0

In [7]:
fc.population

,race_id,horse_id,date
0,202201010101,2020100678,2022-07-23
1,202201010101,2020105743,2022-07-23
2,202201010101,2020102526,2022-07-23
3,202201010101,2020102360,2022-07-23
4,202201010101,2020102535,2022-07-23
...,...,...,...
94108,202310030812,2019102542,2023-09-03
94109,202310030812,2020103754,2023-09-03
94110,202310030812,2020100400,2023-09-03
94111,202310030812,2020103663,2023-09-03


In [8]:
fc.population.race_id.nunique()

6912

In [14]:
fc.horse_results

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses
0,2020104659,2023-01-14,10.0,0.0,2.5,NaN,0,1200,0.0,1.0,16.0
1,2020104659,2022-12-28,10.0,0.0,1.1,NaN,0,1200,0.0,1.0,16.0
2,2020104659,2022-12-04,14.0,0.0,2.9,NaN,0,1200,0.0,1.0,16.0
3,2020104659,2022-11-06,12.0,0.0,1.8,NaN,0,1300,0.0,0.0,16.0
4,2019103180,2022-11-20,6.0,0.0,2.6,NaN,0,1400,2.0,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...
300706,2018105819,2021-03-20,16.0,0.0,2.5,NaN,1,1800,0.0,6.0,16.0
300707,2018105819,2021-01-17,1.0,510.0,0.0,NaN,0,1800,0.0,1.0,16.0
300708,2018105819,2020-12-19,2.0,200.0,0.2,NaN,0,1800,0.0,1.0,16.0
300709,2018105819,2020-11-14,5.0,51.0,1.5,NaN,1,2000,0.0,1.0,13.0


In [23]:
features = fc.create_features()
pd.set_option('display.max_columns', None)
features

,race_id,horse_id,date,jockey_id,trainer_id,owner_id,rank,wakuban,umaban,sex,age,weight,weight_diff,tansho_odds,popularity,impost,race_type,around,course_len,weather,ground_state,race_class,place,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races
0,202201010101,2020100678,2022-07-23,666,1066,356803,1,1,1,1,2,436.0,0.0,1.2,1,54.0,1,0.0,1800.0,1,2,1,1,4.000000,110.000000,4.0,110.0,4.000000,110.000000,4.000000,110.000000
1,202201010101,2020105743,2022-07-23,1091,420,639009,4,2,2,1,2,432.0,-2.0,17.2,4,54.0,1,0.0,1800.0,1,2,1,1,4.500000,81.000000,4.5,81.0,4.500000,81.000000,4.500000,81.000000
2,202201010101,2020102526,2022-07-23,1170,1005,270006,2,3,3,1,2,450.0,-6.0,11.3,3,54.0,1,0.0,1800.0,1,2,1,1,6.000000,0.000000,6.0,0.0,6.000000,0.000000,6.000000,0.000000
3,202201010101,2020102360,2022-07-23,1144,431,970006,5,4,4,1,2,460.0,6.0,28.4,5,54.0,1,0.0,1800.0,1,2,1,1,5.500000,55.000000,5.5,55.0,5.500000,55.000000,5.500000,55.000000
4,202201010101,2020102535,2022-07-23,1140,1005,270006,3,5,5,1,2,412.0,-6.0,4.3,2,54.0,1,0.0,1800.0,1,2,1,1,3.000000,180.000000,3.0,180.0,3.000000,180.000000,3.000000,180.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94108,202310030812,2019102542,2023-09-03,1208,1161,17803,1,6,8,2,4,442.0,-2.0,3.5,1,53.0,1,0.0,2600.0,0,0,2,10,6.666667,0.000000,5.4,64.0,6.000000,44.000000,5.058824,102.823529
94109,202310030812,2020103754,2023-09-03,1116,1104,651031,6,7,9,1,3,468.0,0.0,15.2,8,55.0,1,0.0,2600.0,0,0,2,10,8.000000,40.000000,6.0,145.0,6.125000,90.625000,6.125000,90.625000
94110,202310030812,2020100400,2023-09-03,1193,1092,180800,4,7,10,2,3,458.0,6.0,23.1,9,50.0,1,0.0,2600.0,0,0,2,10,8.000000,0.000000,8.8,0.0,8.800000,0.000000,8.800000,0.000000
94111,202310030812,2020103663,2023-09-03,732,1092,226800,2,8,11,3,3,486.0,-6.0,5.0,3,55.0,1,0.0,2600.0,0,0,2,10,5.666667,61.666667,6.2,97.0,5.666667,123.222222,5.666667,123.222222
